<a href="https://colab.research.google.com/github/tomdkt/test-gpu-llama/blob/main/test_gpu_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cat /etc/os-release
!nvcc --version
import torch
print(torch.cuda.is_available())

True


In [2]:
# upload llama cpp driver
from google.colab import files
uploaded = files.upload()  # selecione o `llama_cpp.zip` no seu computador(google drive)
#!unzip -o llama_cpp.zip -d /usr/local/lib/python3.10/dist-packages/
!unzip -o llama_cpp.zip '*/llama_cpp/*' -d temp_llama
!cp -r temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp /usr/local/lib/python3.11/dist-packages/


!ls /usr/local/lib/python3.10/dist-packages/llama_cpp


import sys
sys.path.append("/usr/local/lib/python3.11/dist-packages")

import llama_cpp
print(llama_cpp.__file__)


MessageError: TypeError: Cannot read properties of undefined (reading 'next')

In [ ]:
from llama_cpp import llama_cpp

print(llama_cpp.llama_supports_gpu_offload())

True


In [ ]:
# compilar tudo do zero. Leva 20 min
# !pip cache purge  # or pip cache remove llama-cpp-python
# !pip uninstall llama-cpp-python
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install llama-cpp-python

# download llama driver
# import os
# import llama_cpp

# lib_path = os.path.dirname(llama_cpp.__file__)
# zip_path = "/content/llama_cpp.zip"
# print(f"Zipping from: {lib_path}")

# # 2. Cria o zip da pasta
# !zip -r /content/llama_cpp.zip "$lib_path"

# # 3. (Opcional) Faz download
# from google.colab import files
# files.download(zip_path)


Files removed: 40
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 39.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=53321240 sha256=9664a3592fe8a928561270a7943e28e54011cdfbcd3d7e9cf6cc4b8fd03ade03
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
	filename="gemma-3-4b-it-q4_0.gguf",
  n_gpu_layers=-1
)

In [ ]:
from IPython.display import display, HTML

response = llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "oi!!! como vc está?"
                }
            ]
        }
    ],
    stream=True
)

# Accumulate the full response
full_response = ""

# Display initial HTML structure for dynamic updates
display(HTML("<b>Resposta:</b> <div id='response'></div><br><br><br>"))

for chunk in response:
    content = chunk["choices"][0]["delta"].get("content", "")
    #full_response += content

    # Update the HTML content with the latest response chunk
    # Using `.format()` instead of f-strings to avoid backslash issues
    display(HTML("<script>document.getElementById('response').innerHTML += '{}';</script>".format(content.replace('`', '\\`'))), display_id='response_update')


Llama.generate: 15 prefix-match hit, remaining 1 prompt tokens to eval


llama_perf_context_print:        load time =     217.35 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =     426.08 ms /    19 runs   (   22.43 ms per token,    44.59 tokens per second)
llama_perf_context_print:       total time =     478.75 ms /    20 tokens
